# 手写识别回顾

In [11]:
from numpy import *
from time import sleep

In [12]:
def loadDataSet(fileName):
    dataMat = []; labelMat = []
    fr = open(fileName)
    for line in fr.readlines():
        lineArr = line.strip().split('\t')
        dataMat.append([float(lineArr[0]), float(lineArr[1])])
        labelMat.append(float(lineArr[2]))
    return dataMat,labelMat

def selectJrand(i,m):
    j=i #we want to select any J not equal to i
    while (j==i):
        j = int(random.uniform(0,m))
    return j

def clipAlpha(aj,H,L):
    if aj > H: 
        aj = H
    if L > aj:
        aj = L
    return aj

## 定义核转换函数

In [13]:
def kernelTrans(X, A, kTup): #calc the kernel or transform data to a higher dimensional space
    m,n = shape(X)
    K = mat(zeros((m,1)))
    if kTup[0]=='lin': K = X * A.T   #linear kernel
    elif kTup[0]=='rbf':
        for j in range(m):
            deltaRow = X[j,:] - A
            K[j] = deltaRow*deltaRow.T
        K = exp(K/(-1*kTup[1]**2)) #divide in NumPy is element-wise not matrix like Matlab
    else: raise NameError('Houston We Have a Problem -- \
    That Kernel is not recognized')
    return K

## 定义完整版 Platt SMO 支持函数

In [14]:
class optStruct:
    def __init__(self,dataMatIn, classLabels, C, toler, kTup):  # Initialize the structure with the parameters 
        self.X = dataMatIn
        self.labelMat = classLabels
        self.C = C
        self.tol = toler
        self.m = shape(dataMatIn)[0]
        self.alphas = mat(zeros((self.m,1)))
        self.b = 0
        self.eCache = mat(zeros((self.m,2))) #first column is valid flag
        self.K = mat(zeros((self.m,self.m)))
        for i in range(self.m):
            self.K[:,i] = kernelTrans(self.X, self.X[i,:], kTup)

def calcEk(oS, k):
    fXk = float(multiply(oS.alphas,oS.labelMat).T*oS.K[:,k] + oS.b)
    Ek = fXk - float(oS.labelMat[k])
    return Ek
        
def selectJ(i, oS, Ei):         #this is the second choice -heurstic, and calcs Ej
    maxK = -1; maxDeltaE = 0; Ej = 0
    oS.eCache[i] = [1,Ei]  #set valid #choose the alpha that gives the maximum delta E
    validEcacheList = nonzero(oS.eCache[:,0].A)[0]
    if (len(validEcacheList)) > 1:
        for k in validEcacheList:   #loop through valid Ecache values and find the one that maximizes delta E
            if k == i: continue #don't calc for i, waste of time
            Ek = calcEk(oS, k)
            deltaE = abs(Ei - Ek)
            if (deltaE > maxDeltaE):
                maxK = k; maxDeltaE = deltaE; Ej = Ek
        return maxK, Ej
    else:   #in this case (first time around) we don't have any valid eCache values
        j = selectJrand(i, oS.m)
        Ej = calcEk(oS, j)
    return j, Ej

def updateEk(oS, k):#after any alpha has changed update the new value in the cache
    Ek = calcEk(oS, k)
    oS.eCache[k] = [1,Ek]

## 定义完整版 Platt SMO 算法优化例程

In [ ]:
def innerL(i, oS):
    Ei = calcEk(oS, i)
    if ((oS.labelMat[i]*Ei < -oS.tol) and (oS.alphas[i] < oS.C)) or ((oS.labelMat[i]*Ei > oS.tol) and (oS.alphas[i] > 0)):
        j,Ej = selectJ(i, oS, Ei) #this has been changed from selectJrand
        alphaIold = oS.alphas[i].copy(); alphaJold = oS.alphas[j].copy();
        if (oS.labelMat[i] != oS.labelMat[j]):
            L = max(0, oS.alphas[j] - oS.alphas[i])
            H = min(oS.C, oS.C + oS.alphas[j] - oS.alphas[i])
        else:
            L = max(0, oS.alphas[j] + oS.alphas[i] - oS.C)
            H = min(oS.C, oS.alphas[j] + oS.alphas[i])
        if L==H: print("L==H"); return 0
        eta = 2.0 * oS.K[i,j] - oS.K[i,i] - oS.K[j,j] #changed for kernel
        if eta >= 0: print("eta>=0"); return 0
        oS.alphas[j] -= oS.labelMat[j]*(Ei - Ej)/eta
        oS.alphas[j] = clipAlpha(oS.alphas[j],H,L)
        updateEk(oS, j) #added this for the Ecache
        if (abs(oS.alphas[j] - alphaJold) < 0.00001): print("j not moving enough"); return 0
        oS.alphas[i] += oS.labelMat[j]*oS.labelMat[i]*(alphaJold - oS.alphas[j])#update i by the same amount as j
        updateEk(oS, i) #added this for the Ecache                    #the update is in the oppostie direction
        b1 = oS.b - Ei- oS.labelMat[i]*(oS.alphas[i]-alphaIold)*oS.K[i,i] - oS.labelMat[j]*(oS.alphas[j]-alphaJold)*oS.K[i,j]
        b2 = oS.b - Ej- oS.labelMat[i]*(oS.alphas[i]-alphaIold)*oS.K[i,j]- oS.labelMat[j]*(oS.alphas[j]-alphaJold)*oS.K[j,j]
        if (0 < oS.alphas[i]) and (oS.C > oS.alphas[i]): oS.b = b1
        elif (0 < oS.alphas[j]) and (oS.C > oS.alphas[j]): oS.b = b2
        else: oS.b = (b1 + b2)/2.0
        return 1
    else: return 0

## 定义完整版 Platt SMO 的外循环代码

In [ ]:
def smoP(dataMatIn, classLabels, C, toler, maxIter,kTup=('lin', 0)):    #full Platt SMO
    oS = optStruct(mat(dataMatIn),mat(classLabels).transpose(),C,toler, kTup)
    iter = 0
    entireSet = True; alphaPairsChanged = 0
    while (iter < maxIter) and ((alphaPairsChanged > 0) or (entireSet)):
        alphaPairsChanged = 0
        if entireSet:   #go over all
            for i in range(oS.m):        
                alphaPairsChanged += innerL(i,oS)
                print("fullSet, iter: %d i:%d, pairs changed %d" % (iter,i,alphaPairsChanged))
            iter += 1
        else:#go over non-bound (railed) alphas
            nonBoundIs = nonzero((oS.alphas.A > 0) * (oS.alphas.A < C))[0]
            for i in nonBoundIs:
                alphaPairsChanged += innerL(i,oS)
                print("non-bound, iter: %d i:%d, pairs changed %d" % (iter,i,alphaPairsChanged))
            iter += 1
        if entireSet: entireSet = False #toggle entire set loop
        elif (alphaPairsChanged == 0): entireSet = True  
        print("iteration number: %d" % iter)
    return oS.b,oS.alphas

## 手写数字识别回顾

In [ ]:
def img2vector(filename):
    returnVect = zeros((1,1024))
    fr = open(filename)
    for i in range(32):
        lineStr = fr.readline()
        for j in range(32):
            returnVect[0,32*i+j] = int(lineStr[j])
    return returnVect

def loadImages(dirName):
    from os import listdir
    hwLabels = []
    trainingFileList = listdir(dirName)           #load the training set
    m = len(trainingFileList)
    trainingMat = zeros((m,1024))
    for i in range(m):
        fileNameStr = trainingFileList[i]
        fileStr = fileNameStr.split('.')[0]     #take off .txt
        classNumStr = int(fileStr.split('_')[0])
        if classNumStr == 9: hwLabels.append(-1)
        else: hwLabels.append(1)
        trainingMat[i,:] = img2vector('%s/%s' % (dirName, fileNameStr))
    return trainingMat, hwLabels    

def testDigits(kTup=('rbf', 10)):
    dataArr,labelArr = loadImages('trainingDigits')
    b,alphas = smoP(dataArr, labelArr, 200, 0.0001, 10000, kTup)
    datMat=mat(dataArr); labelMat = mat(labelArr).transpose()
    svInd=nonzero(alphas.A>0)[0]
    sVs=datMat[svInd] 
    labelSV = labelMat[svInd];
    print("there are %d Support Vectors" % shape(sVs)[0])
    m,n = shape(datMat)
    errorCount = 0
    for i in range(m):
        kernelEval = kernelTrans(sVs,datMat[i,:],kTup)
        predict=kernelEval.T * multiply(labelSV,alphas[svInd]) + b
        if sign(predict)!=sign(labelArr[i]): errorCount += 1
    print("the training error rate is: %f" % (float(errorCount)/m))
    dataArr,labelArr = loadImages('testDigits')
    errorCount = 0
    datMat=mat(dataArr); labelMat = mat(labelArr).transpose()
    m,n = shape(datMat)
    for i in range(m):
        kernelEval = kernelTrans(sVs,datMat[i,:],kTup)
        predict=kernelEval.T * multiply(labelSV,alphas[svInd]) + b
        if sign(predict)!=sign(labelArr[i]): errorCount += 1    
    print("the test error rate is: %f" % (float(errorCount)/m)) 

In [ ]:
testDigits(('rbf', 20))

L==H
fullSet, iter: 0 i:0, pairs changed 0
fullSet, iter: 0 i:1, pairs changed 1
fullSet, iter: 0 i:2, pairs changed 2
fullSet, iter: 0 i:3, pairs changed 3
fullSet, iter: 0 i:4, pairs changed 4
fullSet, iter: 0 i:5, pairs changed 4
fullSet, iter: 0 i:6, pairs changed 4
fullSet, iter: 0 i:7, pairs changed 5
fullSet, iter: 0 i:8, pairs changed 6
fullSet, iter: 0 i:9, pairs changed 7
fullSet, iter: 0 i:10, pairs changed 7
fullSet, iter: 0 i:11, pairs changed 7
fullSet, iter: 0 i:12, pairs changed 7
fullSet, iter: 0 i:13, pairs changed 8
fullSet, iter: 0 i:14, pairs changed 9
fullSet, iter: 0 i:15, pairs changed 9
fullSet, iter: 0 i:16, pairs changed 9
fullSet, iter: 0 i:17, pairs changed 10
fullSet, iter: 0 i:18, pairs changed 11
fullSet, iter: 0 i:19, pairs changed 11
j not moving enough
fullSet, iter: 0 i:20, pairs changed 11
L==H
fullSet, iter: 0 i:21, pairs changed 11
L==H
fullSet, iter: 0 i:22, pairs changed 11
fullSet, iter: 0 i:23, pairs changed 11
fullSet, iter: 0 i:24, pairs cha

L==H
fullSet, iter: 0 i:201, pairs changed 11
L==H
fullSet, iter: 0 i:202, pairs changed 11
L==H
fullSet, iter: 0 i:203, pairs changed 11
L==H
fullSet, iter: 0 i:204, pairs changed 11
L==H
fullSet, iter: 0 i:205, pairs changed 11
L==H
fullSet, iter: 0 i:206, pairs changed 11
L==H
fullSet, iter: 0 i:207, pairs changed 11
L==H
fullSet, iter: 0 i:208, pairs changed 11
L==H
fullSet, iter: 0 i:209, pairs changed 11
L==H
fullSet, iter: 0 i:210, pairs changed 11
L==H
fullSet, iter: 0 i:211, pairs changed 11
L==H
fullSet, iter: 0 i:212, pairs changed 11
L==H
fullSet, iter: 0 i:213, pairs changed 11
L==H
fullSet, iter: 0 i:214, pairs changed 11
L==H
fullSet, iter: 0 i:215, pairs changed 11
L==H
fullSet, iter: 0 i:216, pairs changed 11
L==H
fullSet, iter: 0 i:217, pairs changed 11
L==H
fullSet, iter: 0 i:218, pairs changed 11
L==H
fullSet, iter: 0 i:219, pairs changed 11
L==H
fullSet, iter: 0 i:220, pairs changed 11
L==H
fullSet, iter: 0 i:221, pairs changed 11
L==H
fullSet, iter: 0 i:222, pairs

fullSet, iter: 0 i:380, pairs changed 11
L==H
fullSet, iter: 0 i:381, pairs changed 11
L==H
fullSet, iter: 0 i:382, pairs changed 11
L==H
fullSet, iter: 0 i:383, pairs changed 11
L==H
fullSet, iter: 0 i:384, pairs changed 11
L==H
fullSet, iter: 0 i:385, pairs changed 11
L==H
fullSet, iter: 0 i:386, pairs changed 11
L==H
fullSet, iter: 0 i:387, pairs changed 11
L==H
fullSet, iter: 0 i:388, pairs changed 11
L==H
fullSet, iter: 0 i:389, pairs changed 11
L==H
fullSet, iter: 0 i:390, pairs changed 11
L==H
fullSet, iter: 0 i:391, pairs changed 11
L==H
fullSet, iter: 0 i:392, pairs changed 11
L==H
fullSet, iter: 0 i:393, pairs changed 11
L==H
fullSet, iter: 0 i:394, pairs changed 11
L==H
fullSet, iter: 0 i:395, pairs changed 11
L==H
fullSet, iter: 0 i:396, pairs changed 11
L==H
fullSet, iter: 0 i:397, pairs changed 11
L==H
fullSet, iter: 0 i:398, pairs changed 11
L==H
fullSet, iter: 0 i:399, pairs changed 11
L==H
fullSet, iter: 0 i:400, pairs changed 11
L==H
fullSet, iter: 0 i:401, pairs chan

L==H
fullSet, iter: 0 i:561, pairs changed 11
L==H
fullSet, iter: 0 i:562, pairs changed 11
L==H
fullSet, iter: 0 i:563, pairs changed 11
L==H
fullSet, iter: 0 i:564, pairs changed 11
L==H
fullSet, iter: 0 i:565, pairs changed 11
L==H
fullSet, iter: 0 i:566, pairs changed 11
L==H
fullSet, iter: 0 i:567, pairs changed 11
L==H
fullSet, iter: 0 i:568, pairs changed 11
L==H
fullSet, iter: 0 i:569, pairs changed 11
L==H
fullSet, iter: 0 i:570, pairs changed 11
L==H
fullSet, iter: 0 i:571, pairs changed 11
L==H
fullSet, iter: 0 i:572, pairs changed 11
L==H
fullSet, iter: 0 i:573, pairs changed 11
L==H
fullSet, iter: 0 i:574, pairs changed 11
L==H
fullSet, iter: 0 i:575, pairs changed 11
L==H
fullSet, iter: 0 i:576, pairs changed 11
L==H
fullSet, iter: 0 i:577, pairs changed 11
L==H
fullSet, iter: 0 i:578, pairs changed 11
L==H
fullSet, iter: 0 i:579, pairs changed 11
L==H
fullSet, iter: 0 i:580, pairs changed 11
L==H
fullSet, iter: 0 i:581, pairs changed 11
L==H
fullSet, iter: 0 i:582, pairs

L==H
fullSet, iter: 0 i:743, pairs changed 11
L==H
fullSet, iter: 0 i:744, pairs changed 11
L==H
fullSet, iter: 0 i:745, pairs changed 11
L==H
fullSet, iter: 0 i:746, pairs changed 11
L==H
fullSet, iter: 0 i:747, pairs changed 11
L==H
fullSet, iter: 0 i:748, pairs changed 11
L==H
fullSet, iter: 0 i:749, pairs changed 11
L==H
fullSet, iter: 0 i:750, pairs changed 11
L==H
fullSet, iter: 0 i:751, pairs changed 11
L==H
fullSet, iter: 0 i:752, pairs changed 11
L==H
fullSet, iter: 0 i:753, pairs changed 11
L==H
fullSet, iter: 0 i:754, pairs changed 11
L==H
fullSet, iter: 0 i:755, pairs changed 11
L==H
fullSet, iter: 0 i:756, pairs changed 11
L==H
fullSet, iter: 0 i:757, pairs changed 11
L==H
fullSet, iter: 0 i:758, pairs changed 11
L==H
fullSet, iter: 0 i:759, pairs changed 11
L==H
fullSet, iter: 0 i:760, pairs changed 11
L==H
fullSet, iter: 0 i:761, pairs changed 11
L==H
fullSet, iter: 0 i:762, pairs changed 11
L==H
fullSet, iter: 0 i:763, pairs changed 11
L==H
fullSet, iter: 0 i:764, pairs

...

L==H
fullSet, iter: 7 i:741, pairs changed 0
L==H
fullSet, iter: 7 i:742, pairs changed 0
L==H
fullSet, iter: 7 i:743, pairs changed 0
j not moving enough
fullSet, iter: 7 i:744, pairs changed 0
L==H
fullSet, iter: 7 i:745, pairs changed 0
j not moving enough
fullSet, iter: 7 i:746, pairs changed 0
j not moving enough
fullSet, iter: 7 i:747, pairs changed 0
L==H
fullSet, iter: 7 i:748, pairs changed 0
fullSet, iter: 7 i:749, pairs changed 0
L==H
fullSet, iter: 7 i:750, pairs changed 0
fullSet, iter: 7 i:751, pairs changed 0
L==H
fullSet, iter: 7 i:752, pairs changed 0
j not moving enough
fullSet, iter: 7 i:753, pairs changed 0
L==H
fullSet, iter: 7 i:754, pairs changed 0
j not moving enough
fullSet, iter: 7 i:755, pairs changed 0
j not moving enough
fullSet, iter: 7 i:756, pairs changed 0
fullSet, iter: 7 i:757, pairs changed 0
L==H
fullSet, iter: 7 i:758, pairs changed 0
L==H
fullSet, iter: 7 i:759, pairs changed 0
j not moving enough
fullSet, iter: 7 i:760, pairs changed 0
fullSet, i

L==H
fullSet, iter: 7 i:939, pairs changed 0
fullSet, iter: 7 i:940, pairs changed 0
L==H
fullSet, iter: 7 i:941, pairs changed 0
fullSet, iter: 7 i:942, pairs changed 0
fullSet, iter: 7 i:943, pairs changed 0
fullSet, iter: 7 i:944, pairs changed 0
fullSet, iter: 7 i:945, pairs changed 0
fullSet, iter: 7 i:946, pairs changed 0
fullSet, iter: 7 i:947, pairs changed 0
fullSet, iter: 7 i:948, pairs changed 0
fullSet, iter: 7 i:949, pairs changed 0
fullSet, iter: 7 i:950, pairs changed 0
fullSet, iter: 7 i:951, pairs changed 0
fullSet, iter: 7 i:952, pairs changed 0
fullSet, iter: 7 i:953, pairs changed 0
fullSet, iter: 7 i:954, pairs changed 0
fullSet, iter: 7 i:955, pairs changed 0
fullSet, iter: 7 i:956, pairs changed 0
fullSet, iter: 7 i:957, pairs changed 0
fullSet, iter: 7 i:958, pairs changed 0
fullSet, iter: 7 i:959, pairs changed 0
fullSet, iter: 7 i:960, pairs changed 0
j not moving enough
fullSet, iter: 7 i:961, pairs changed 0
fullSet, iter: 7 i:962, pairs changed 0
L==H
fullS

L==H
fullSet, iter: 7 i:1129, pairs changed 0
L==H
fullSet, iter: 7 i:1130, pairs changed 0
L==H
fullSet, iter: 7 i:1131, pairs changed 0
L==H
fullSet, iter: 7 i:1132, pairs changed 0
L==H
fullSet, iter: 7 i:1133, pairs changed 0
L==H
fullSet, iter: 7 i:1134, pairs changed 0
L==H
fullSet, iter: 7 i:1135, pairs changed 0
L==H
fullSet, iter: 7 i:1136, pairs changed 0
fullSet, iter: 7 i:1137, pairs changed 0
fullSet, iter: 7 i:1138, pairs changed 0
fullSet, iter: 7 i:1139, pairs changed 0
fullSet, iter: 7 i:1140, pairs changed 0
fullSet, iter: 7 i:1141, pairs changed 0
fullSet, iter: 7 i:1142, pairs changed 0
L==H
fullSet, iter: 7 i:1143, pairs changed 0
fullSet, iter: 7 i:1144, pairs changed 0
fullSet, iter: 7 i:1145, pairs changed 0
fullSet, iter: 7 i:1146, pairs changed 0
fullSet, iter: 7 i:1147, pairs changed 0
fullSet, iter: 7 i:1148, pairs changed 0
L==H
fullSet, iter: 7 i:1149, pairs changed 0
L==H
fullSet, iter: 7 i:1150, pairs changed 0
fullSet, iter: 7 i:1151, pairs changed 0
fu

j not moving enough
fullSet, iter: 7 i:1358, pairs changed 0
L==H
fullSet, iter: 7 i:1359, pairs changed 0
fullSet, iter: 7 i:1360, pairs changed 0
L==H
fullSet, iter: 7 i:1361, pairs changed 0
fullSet, iter: 7 i:1362, pairs changed 0
fullSet, iter: 7 i:1363, pairs changed 0
fullSet, iter: 7 i:1364, pairs changed 0
fullSet, iter: 7 i:1365, pairs changed 0
fullSet, iter: 7 i:1366, pairs changed 0
L==H
fullSet, iter: 7 i:1367, pairs changed 0
fullSet, iter: 7 i:1368, pairs changed 0
fullSet, iter: 7 i:1369, pairs changed 0
L==H
fullSet, iter: 7 i:1370, pairs changed 0
fullSet, iter: 7 i:1371, pairs changed 0
L==H
fullSet, iter: 7 i:1372, pairs changed 0
fullSet, iter: 7 i:1373, pairs changed 0
j not moving enough
fullSet, iter: 7 i:1374, pairs changed 0
fullSet, iter: 7 i:1375, pairs changed 0
fullSet, iter: 7 i:1376, pairs changed 0
fullSet, iter: 7 i:1377, pairs changed 0
fullSet, iter: 7 i:1378, pairs changed 0
fullSet, iter: 7 i:1379, pairs changed 0
fullSet, iter: 7 i:1380, pairs ch

L==H
fullSet, iter: 7 i:1551, pairs changed 0
fullSet, iter: 7 i:1552, pairs changed 0
fullSet, iter: 7 i:1553, pairs changed 0
fullSet, iter: 7 i:1554, pairs changed 0
L==H
fullSet, iter: 7 i:1555, pairs changed 0
fullSet, iter: 7 i:1556, pairs changed 0
fullSet, iter: 7 i:1557, pairs changed 0
fullSet, iter: 7 i:1558, pairs changed 0
fullSet, iter: 7 i:1559, pairs changed 0
fullSet, iter: 7 i:1560, pairs changed 0
fullSet, iter: 7 i:1561, pairs changed 0
fullSet, iter: 7 i:1562, pairs changed 0
fullSet, iter: 7 i:1563, pairs changed 0
fullSet, iter: 7 i:1564, pairs changed 0
L==H
fullSet, iter: 7 i:1565, pairs changed 0
fullSet, iter: 7 i:1566, pairs changed 0
fullSet, iter: 7 i:1567, pairs changed 0
fullSet, iter: 7 i:1568, pairs changed 0
L==H
fullSet, iter: 7 i:1569, pairs changed 0
L==H
fullSet, iter: 7 i:1570, pairs changed 0
fullSet, iter: 7 i:1571, pairs changed 0
fullSet, iter: 7 i:1572, pairs changed 0
fullSet, iter: 7 i:1573, pairs changed 0
fullSet, iter: 7 i:1574, pairs c